# Initial Data Load

We'll load the data first and check the three types of files. 

In [1]:
import os

In [2]:
import pandas as pd
import sklearn
import scipy.sparse as sp
import numpy as np

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [3]:
DATA_ROOT = "/ysm-gpfs/project/smz25/CHUD/"

In [4]:
var_annotations = DATA_ROOT + "UKBB_CHIP-somVariants.filtered_rare_disruptive_LOF.annotated.bgz"
annotations = pd.read_csv(var_annotations, delimiter='\t', compression='gzip')
# print(annotations.columns)
print("Variant Annotation File Shape: " , annotations.shape)
print(annotations.dtypes)
# print(annotations.head())

# print("\n\n")


/gpfs/ysm/project/rtaylor/vs428/conda_envs/chudenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Variant Annotation File Shape:  (1409376, 39)
f0                       object
locus                    object
alleles                  object
all_transcript_terms     object
Consequence              object
Gene                     object
hgvsp                    object
hgvsc                    object
SIFT                     object
PolyPhen                 object
LOF_LOFTEE               object
clin_sig                 object
phenotype_or_disease     object
pubmed                   object
SOMATIC                  object
metasvm                  object
Leukemia                 object
TOPMed_CHIPVar           object
LeukemiaGene            float64
nfe_AF_g                float64
fin_AF_g                float64
oth_AF_g                float64
asj_AF_g                float64
afr_AF_g                float64
eas_AF_g                float64
sas_AF_g                float64
amr_AF_g                float64
raw_AF_g                float64
adj_AF_g                float64
nfe_AF_e                fl

In [5]:
sample_file = DATA_ROOT + "QC_v1/Filtered_SomaticCalls_v1/2657375_23173_0_0-filtered.vcf.gz.1343.GTs.bgz"
samples = pd.read_csv(sample_file, delimiter='\t', compression='gzip')
# # print(samples.columns)
# print("Sample File Shape: " , samples.shape)
# print(samples.dtypes)
samples.head()

# print("\n\n")

,v,NumAltAlleles,VAF,TLOD,GERMQ,STR,AD_alt,AD_ref,s,AD,AF,DP,F1R2,F2R1,GT,SB,Binomial_Prob
0,chr10:102120214:G:T,1,0.547,111.30,6,False,33.0,27.0,UKB_5272548_0230807599,"[27,33]",[0.547],60,"[9,15]","[18,17]",0/1,"[12,15,14,19]",0.81685
1,chr10:104033244:A:T,1,0.563,113.35,6,False,39.0,30.0,UKB_5272548_0230807599,"[30,39]",[0.563],69,"[11,19]","[19,20]",0/1,"[18,12,21,18]",0.88580
2,chr10:104033415:T:G,1,0.977,143.99,93,False,40.0,0.0,UKB_5272548_0230807599,"[0,40]",[0.977],40,"[0,11]","[0,29]",0/1,"[0,0,7,33]",1.00000
3,chr10:104314021:C:T,1,0.450,58.91,10,False,17.0,21.0,UKB_5272548_0230807599,"[21,17]",[0.45],38,"[14,10]","[7,7]",0/1,"[10,11,9,8]",0.31355
4,chr10:110507064:GC:AT,1,0.984,251.78,93,False,57.0,0.0,UKB_5272548_0230807599,"[0,57]",[0.984],57,"[0,32]","[0,24]",0/1,"[0,0,32,25]",1.00000


In [6]:
phenotype_file = DATA_ROOT + "ukbb_PhenoFile.ForClassProj.QCed.txt.gz"
phenotypes = pd.read_csv(phenotype_file, delimiter='\t', compression='gzip')
# print(phenotypes.columns)
print("Phenotype File Shape: " , phenotypes.shape)
print(phenotypes.dtypes)
# print(phenotypes.head())

Phenotype File Shape:  (391396, 73)
id                                         int64
Sex                                       object
Sex_numeric                                int64
age                                        int64
SmokingStatus                            float64
SmokingStatusv2                           object
ever_smoked                                int64
age2                                       int64
genotyping_array                          object
PC1                                      float64
PC2                                      float64
PC3                                      float64
PC4                                      float64
PC5                                      float64
PC6                                      float64
PC7                                      float64
PC8                                      float64
PC9                                      float64
PC10                                     float64
Coronary_Artery_Disease_SOFT     

Fix Phenotype file types

In [7]:
annotations["LeukemiaGene"].unique()

array([nan,  1.])

In [8]:
phenotypes.drop(inplace=True, columns=["Sex", "SmokingStatusv2", "genotyping_array", "transcriptOI" ])# , "AD"])
phenotypes.dtypes

id                                         int64
Sex_numeric                                int64
age                                        int64
SmokingStatus                            float64
ever_smoked                                int64
age2                                       int64
PC1                                      float64
PC2                                      float64
PC3                                      float64
PC4                                      float64
PC5                                      float64
PC6                                      float64
PC7                                      float64
PC8                                      float64
PC9                                      float64
PC10                                     float64
Coronary_Artery_Disease_SOFT               int64
Prev_Coronary_Artery_Disease_SOFT        float64
Incd_Coronary_Artery_Disease_SOFT        float64
Coronary_Artery_Disease_SOFT_FollowUp    float64
Death               

In [9]:
phenotypes[phenotypes['AD'].str.count(",") > 1]

,id,Sex_numeric,age,SmokingStatus,ever_smoked,age2,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Coronary_Artery_Disease_SOFT,Prev_Coronary_Artery_Disease_SOFT,Incd_Coronary_Artery_Disease_SOFT,Coronary_Artery_Disease_SOFT_FollowUp,Death,Prev_Death,Incd_Death,Death_FollowUp,Diabetes_All,Prev_Diabetes_All,Incd_Diabetes_All,Diabetes_All_FollowUp,Diabetes_Type_1,Prev_Diabetes_Type_1,Incd_Diabetes_Type_1,Diabetes_Type_1_FollowUp,Diabetes_Type_2,Prev_Diabetes_Type_2,Incd_Diabetes_Type_2,Diabetes_Type_2_FollowUp,Heart_Failure,Prev_Heart_Failure,Incd_Heart_Failure,Heart_Failure_FollowUp,Hypercholesterolemia,Prev_Hypercholesterolemia,Incd_Hypercholesterolemia,Hypercholesterolemia_FollowUp,Hypertension,Prev_Hypertension,Incd_Hypertension,Hypertension_FollowUp,CHIP_Gene,AD,AF,hasCHIP,AML,Prev_AML,Incd_AML,AML_FollowUp,MPN,Prev_MPN,Incd_MPN,MPN_FollowUp,Platelet count,Platelet crit,Mean platelet volume,Platelet distribution width,All_Pneumonia,AgeAt_All_Pneumonia,Incd_All_Pneumonia,All_Pneumonia_FollowUp,Prev_All_Pneumonia
4656,1059643,0,56,0.0,0,3136,-14.7054,1.28481,-0.640748,1.36859,-1.20854,-0.252736,0.996171,-4.07007,-1.52349,2.41992,0,0.0,0.0,7.589322,0,0.0,0,7.589322,0,0.0,0.0,7.589322,0,0.0,0.0,7.589322,0,0.0,0.0,7.589322,0.0,0.0,0.0,7.589322,0,0.0,0.0,7.589322,0,0.0,0.0,7.589322,DNMT3A,"70,2,2",NaN,1,0.0,0.0,0.0,8.711111,0.0,0.0,0.0,8.711111,265.0,0.216,8.2,16.0,0.0,NaN,0.0,8.711111,0.0


In [10]:
phenotypes['AD'].unique().shape

(812,)

## Merge the Genotype and Phenotype files

We're going to merge the files by creating a sparse matrix with samples as rows and variants as columns (with binary encoding). First we'll need the total list of variants from the filtered annotations file. 

In [11]:
# annotations['TOPMed_CHIPVar'].unique()


In [12]:
from sklearn.feature_extraction.text import CountVectorizer


In [13]:
# get two different subsets of the variants based on annotations
# We also want to filter by binomial prob and VAF, but those are in the sample files. 
# variant_list = annotations[
# #                    (annotations['SOMATIC'].str.contains("1")) | 
#                    (annotations['LeukemiaGene'] == 1.0) | 
#                     annotations['TOPMed_CHIPVar'].notna()
#            ]["f0"]

variant_list = annotations["f0"]


In [14]:
# delete the annotations dataframe, to save space. we're done with it. TODO: remove this
del annotations

In [15]:
sample_file_dir = DATA_ROOT + "QC_v1/Filtered_SomaticCalls_v1/"
sample_file_list = sorted([f for f in os.listdir(sample_file_dir) if f.endswith('.bgz')])

In [16]:
# sample_var_df = pd.DataFrame(columns=variant_list)
# sample_var_df = sp.dok_matrix((len(sample_file_list),len(variant_list)), dtype=np.int8)
row_names = np.array(sample_file_list)
# col_names = np.array(variant_list)

In [17]:
# print(sample_var_df.shape)
# print(row_names.shape)
# print(row_names[:5])
# print(col_names.shape)
# print(col_names[:5])
# sample_file_list[:4]

We loop through each file, extract the sampleID, and add a string of the variants to a list

In [18]:
sample_var_list = []
sample_list = []

variant_list_sample_set = set()
import time

start = time.time()

for idx, file in enumerate(sample_file_list):
    data = pd.read_csv(sample_file_dir+file, delimiter='\t', compression='gzip')
    # get the sample IDs
    data['sampleID'] = file.split('.', 4)[0].split('-')[0].split("_")[0]
    sample_list.append(int(data['sampleID'].iloc[0]))
    # get the variant list for this file
    sample_variant_list = data[(data["Binomial_Prob"] <= 0.01) & (data["VAF"] >= 0.1)]["v"]
    variant_list_sample_set.update(sample_variant_list)
    # add the variants selected as a "sentence"
    # sample_variant_list will include some that aren't a part of our variant list but will be considered OOV
    # when we do our CountVectorizer method
    sample_var_list.append(sample_variant_list.str.cat(sep=' '))

    if (idx % 1000) == 0:
        print(f"{idx} variants: {time.time() - start:.3f} seconds so far.")

        
#     if idx == 1000:
#         break
        
# data['sampleID']


0 variants: 0.027 seconds so far.
1000 variants: 26.473 seconds so far.
2000 variants: 54.750 seconds so far.
3000 variants: 83.527 seconds so far.
4000 variants: 111.928 seconds so far.
5000 variants: 139.964 seconds so far.
6000 variants: 167.544 seconds so far.
7000 variants: 194.748 seconds so far.
8000 variants: 222.319 seconds so far.
9000 variants: 249.545 seconds so far.
10000 variants: 276.574 seconds so far.
11000 variants: 304.078 seconds so far.
12000 variants: 331.818 seconds so far.
13000 variants: 359.745 seconds so far.
14000 variants: 387.866 seconds so far.
15000 variants: 415.808 seconds so far.
16000 variants: 444.007 seconds so far.
17000 variants: 473.323 seconds so far.
18000 variants: 502.147 seconds so far.
19000 variants: 531.768 seconds so far.
20000 variants: 560.452 seconds so far.
21000 variants: 589.266 seconds so far.
22000 variants: 617.897 seconds so far.
23000 variants: 646.674 seconds so far.
24000 variants: 672.878 seconds so far.
25000 variants: 70

In [19]:
# print(data['sampleID'])
variants = np.intersect1d(variant_list, list(variant_list_sample_set))
col_names = variants
# print(variant_list.shape)
# print(len(variant_list_sample_set))
# print(variants.shape)

In [20]:
# sample_list = [int(sample) for sample in sample_list]

In [21]:
# print(sample_var_list[:5])
# 4671920 in phenotypes['id']

In [22]:
vec = CountVectorizer(vocabulary=col_names, binary=True, dtype=np.int8, 
                      lowercase=False, tokenizer=lambda x:x.split(" "))
X = vec.fit_transform(sample_var_list)

We'll quickly save all the files just in case

In [23]:
np.savetxt("sample_var_onehot_rownames_somatic_leuk.txt", sample_file_list, fmt="%s")
np.savetxt("sample_var_onehot_colnames_somatic_leuk.txt", col_names, fmt="%s")
np.savetxt("sample_name_list_somatic_leuk.txt", sample_list, fmt="%d")
sp.save_npz('sample_var_onehot_somatic_leuk.npz', X)

Now we create a dataframe from that data and merge with the phenotype file on sampleID

In [24]:
# print(phenotypes.dtypes)
phenotypes.head()


,id,Sex_numeric,age,SmokingStatus,ever_smoked,age2,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Coronary_Artery_Disease_SOFT,Prev_Coronary_Artery_Disease_SOFT,Incd_Coronary_Artery_Disease_SOFT,Coronary_Artery_Disease_SOFT_FollowUp,Death,Prev_Death,Incd_Death,Death_FollowUp,Diabetes_All,Prev_Diabetes_All,Incd_Diabetes_All,Diabetes_All_FollowUp,Diabetes_Type_1,Prev_Diabetes_Type_1,Incd_Diabetes_Type_1,Diabetes_Type_1_FollowUp,Diabetes_Type_2,Prev_Diabetes_Type_2,Incd_Diabetes_Type_2,Diabetes_Type_2_FollowUp,Heart_Failure,Prev_Heart_Failure,Incd_Heart_Failure,Heart_Failure_FollowUp,Hypercholesterolemia,Prev_Hypercholesterolemia,Incd_Hypercholesterolemia,Hypercholesterolemia_FollowUp,Hypertension,Prev_Hypertension,Incd_Hypertension,Hypertension_FollowUp,CHIP_Gene,AD,AF,hasCHIP,AML,Prev_AML,Incd_AML,AML_FollowUp,MPN,Prev_MPN,Incd_MPN,MPN_FollowUp,Platelet count,Platelet crit,Mean platelet volume,Platelet distribution width,All_Pneumonia,AgeAt_All_Pneumonia,Incd_All_Pneumonia,All_Pneumonia_FollowUp,Prev_All_Pneumonia
0,1000023,1,44,2.0,1,1936,-14.07040,2.83809,-1.709870,-0.782455,-7.935710,-2.176480,-0.688286,-2.59305,4.890120,5.056270,0,0.0,0.0,7.430527,0,0.0,0,7.430527,0,0.0,0.0,7.430527,0,0.0,0.0,7.430527,0,0.0,0.0,7.430527,0.0,0.0,0.0,7.430527,0,0.0,0.0,7.430527,0,0.0,0.0,7.430527,NaN,NaN,NaN,0,0.0,0.0,0.0,8.555555,0.0,0.0,0.0,8.555555,433.6,0.368,8.51,16.15,0.0,NaN,0.0,8.555555,0.0
1,1000037,0,69,1.0,1,4761,-15.00630,4.03881,-1.711670,6.200160,-2.770400,0.809991,1.012190,3.03839,5.836320,-1.955720,0,0.0,0.0,7.066393,0,0.0,0,7.066393,0,0.0,0.0,7.066393,0,0.0,0.0,7.066393,0,0.0,0.0,7.066393,0.0,0.0,0.0,7.066393,0,0.0,0.0,7.066393,0,0.0,0.0,7.066393,NaN,NaN,NaN,0,0.0,0.0,0.0,8.191667,0.0,0.0,0.0,8.191667,221.0,0.174,7.91,16.52,0.0,NaN,0.0,8.191667,0.0
2,1000042,1,66,0.0,0,4356,-12.08720,6.29019,-1.486040,-3.426860,-5.359130,-0.146506,2.788340,-4.25511,-6.282060,-0.862379,0,0.0,0.0,6.258727,0,0.0,0,6.258727,0,0.0,0.0,6.258727,0,0.0,0.0,6.258727,0,0.0,0.0,6.258727,0.0,0.0,0.0,6.258727,0,0.0,0.0,6.258727,0,0.0,0.0,6.258727,NaN,NaN,NaN,0,0.0,0.0,0.0,7.380555,0.0,0.0,0.0,7.380555,288.7,0.236,8.20,16.79,0.0,NaN,0.0,7.380555,0.0
3,1000059,0,54,1.0,1,2916,-9.58886,3.77243,0.274298,-1.357730,1.917950,-1.627780,-0.191782,-1.28348,2.726250,1.320520,0,0.0,0.0,6.743326,0,0.0,0,6.743326,0,0.0,0.0,6.743326,0,0.0,0.0,6.743326,0,0.0,0.0,6.743326,0.0,0.0,0.0,6.743326,0,0.0,0.0,6.743326,0,0.0,0.0,6.743326,NaN,NaN,NaN,0,0.0,0.0,0.0,7.863888,0.0,0.0,0.0,7.863888,370.3,0.336,9.09,16.36,0.0,NaN,0.0,7.863888,0.0
4,1000061,0,47,1.0,1,2209,-9.83059,3.67183,-0.557439,1.851660,-0.349985,-3.238940,-2.291060,-1.76164,-0.691142,1.272970,0,0.0,0.0,8.024641,0,0.0,0,8.024641,0,0.0,0.0,8.024641,0,0.0,0.0,8.024641,0,0.0,0.0,8.024641,0.0,0.0,0.0,8.024641,0,0.0,0.0,8.024641,0,0.0,0.0,8.024641,NaN,NaN,NaN,0,0.0,0.0,0.0,9.152778,0.0,0.0,0.0,9.152778,208.0,0.230,11.10,17.00,0.0,NaN,0.0,9.152778,0.0


In [25]:
# any(phenotypes['id'] == 2635077)
del sample_var_list

In [26]:
print(X.shape)
sample_variant_df = pd.DataFrame(data=X.toarray(), columns=col_names)
sample_variant_df.head()

(49730, 196555)


,chr10:100048756:A:T,chr10:100048781:C:T,chr10:100048878:T:C,chr10:100054349:G:T,chr10:100057091:G:T,chr10:1000687:C:A,chr10:1000741:C:A,chr10:100152201:G:T,chr10:100152353:C:T,chr10:100154941:T:A,chr10:100164073:G:T,chr10:100164091:C:A,chr10:100167351:A:T,chr10:100167364:T:C,chr10:100167397:C:T,chr10:100167400:G:T,chr10:100174209:T:G,chr10:100174221:C:T,chr10:100178135:G:T,chr10:100178145:C:T,chr10:100178168:C:T,chr10:100183758:G:A,chr10:100183758:G:T,chr10:100183763:G:T,chr10:100183780:G:T,chr10:100183797:G:T,chr10:100185512:A:C,chr10:100185517:T:C,chr10:100185521:A:G,chr10:100193367:G:T,chr10:100194022:T:C,chr10:100194523:T:G,chr10:100204543:G:T,chr10:100204584:T:A,chr10:100204650:G:T,chr10:100205089:G:T,chr10:100207324:A:T,chr10:100209716:G:T,chr10:100218733:G:T,chr10:100219079:G:T,chr10:100220677:C:A,chr10:100235665:A:T,chr10:100235673:A:C,chr10:100235716:C:A,chr10:100235731:C:A,chr10:100238158:G:T,chr10:100238229:G:T,chr10:100250256:T:A,chr10:100256286:A:T,chr10:100260321:T:A,chr10:100280228:T:TATTTCTGGTTTAAGTC,chr10:100286116:G:T,chr10:100288446:C:A,chr10:100290112:T:C,chr10:100294575:T:G,chr10:100294619:G:T,chr10:100294635:A:G,chr10:100295024:T:A,chr10:100295084:G:T,chr10:100295104:A:T,chr10:100295124:C:A,chr10:100296996:T:C,chr10:100297004:G:T,chr10:100297080:A:T,chr10:100298561:C:T,chr10:100299623:G:A,chr10:100299651:ATG:A,chr10:100348333:C:A,chr10:100352414:A:C,chr10:100356765:G:A,chr10:100360740:G:A,chr10:100360745:C:A,chr10:100360754:C:A,chr10:100360757:C:A,chr10:100479052:G:C,chr10:100479054:C:A,chr10:100479085:G:T,chr10:100479956:G:A,chr10:100481078:GATTTTGAT:G,chr10:100482500:T:C,chr10:100489270:T:TGGAAC,chr10:100490203:G:A,chr10:100495509:G:T,chr10:100497275:G:C,chr10:100498070:G:C,chr10:100498133:C:A,chr10:100498774:CAGAG:C,chr10:100499527:CTT:C,chr10:100502317:G:C,chr10:100502363:G:T,chr10:100502486:T:A,chr10:100505396:T:C,chr10:100508001:G:T,chr10:100509089:G:T,chr10:100509441:C:T,chr10:100516216:G:T,chr10:100516872:A:G,chr10:100516951:A:G,chr10:100529386:C:T,chr10:100529508:T:C,...,chrX:9967973:C:T,chrY:12720738:G:A,chrY:12726686:A:T,chrY:12736008:C:T,chrY:12736197:C:T,chrY:12736418:C:T,chrY:12757307:G:A,chrY:12776724:G:GTCACC,chrY:12776726:TGGTGA:T,chrY:12776750:T:A,chrY:12776857:C:A,chrY:12778756:G:T,chrY:12778757:T:A,chrY:12779607:A:T,chrY:12791497:A:G,chrY:12811656:G:T,chrY:12812930:C:A,chrY:12818578:C:A,chrY:12833759:GT:G,chrY:12840013:C:A,chrY:12846484:C:A,chrY:12856710:TG:T,chrY:12915153:C:A,chrY:12916372:C:A,chrY:12916636:T:G,chrY:13251131:G:T,chrY:13297706:C:T,chrY:13305539:A:T,chrY:13306084:T:A,chrY:13323637:C:T,chrY:13335764:T:TAAATCCACTGAGCAGAGAAGCATAAA,chrY:13357905:G:T,chrY:13359836:G:T,chrY:13359866:G:T,chrY:13414758:G:T,chrY:14622338:C:A,chrY:14622565:T:A,chrY:14723253:C:A,chrY:14841079:C:A,chrY:19707199:G:T,chrY:19707221:C:A,chrY:19707481:C:T,chrY:19707541:G:T,chrY:19707556:T:A,chrY:19707599:C:T,chrY:19707679:G:T,chrY:19708011:T:A,chrY:19708259:G:T,chrY:19715425:GT:G,chrY:19715428:G:GA,chrY:19715828:G:T,chrY:19721129:C:A,chrY:19721204:G:T,chrY:19735426:T:A,chrY:19735440:G:T,chrY:19735490:G:T,chrY:19741770:C:A,chrY:20584490:C:A,chrY:20589577:T:A,chrY:20759929:T:A,chrY:20759977:T:C,chrY:20760007:G:A,chrY:20760049:G:A,chrY:20761355:C:A,chrY:20768875:C:A,chrY:20768878:G:A,chrY:20779583:G:C,chrY:20779668:TA:CC,chrY:21383758:A:T,chrY:21903904:T:A,chrY:2787066:G:T,chrY:2787253:C:A,chrY:2787314:T:A,chrY:2787331:G:T,chrY:2787416:G:T,chrY:2787438:C:T,chrY:2787453:T:A,chrY:2787500:A:T,chrY:2787549:G:T,chrY:2787562:G:T,chrY:2787596:G:T,chrY:2842227:A:ATACACCCGTTAGTTTGT,chrY:2844241:C:A,chrY:2854612:A:T,chrY:2961644:C:A,chrY:2979303:C:A,chrY:2979396:T:A,chrY:2979427:A:T,chrY:2979577:G:T,chrY:2979918:C:A,chrY:5056943:C:A,chrY:5737608:C:A,chrY:6868264:G:A,chrY:6868348:CTGGG:C,chrY:6868354:T:TCCCA,chrY:6868356:T:A,chrY:6868360:G:T,chrY:7070862:G:T,chrY:7080808:C:A,chrY:7085951:C:A
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
len(sample_list)

49730

In [28]:
sample_variant_df['sampleID'] = sample_list
del sample_list
# phenotypes.dtypes
# sample_variant_df[sample_variant_df['sampleID'] == 3178890].sum(axis=1)

The code below doesn't work because we would need ~70 GB to hold the dense matrix, before the merge. Instead we will write to file as we loop through each sample vcf file, or we'll store everything as a sparse numeric scipy array

Update Apr. 23, 2019: Works now since we filtered variants above. 

In [29]:
merged_inner = sample_variant_df.merge(right=phenotypes, left_on='sampleID', right_on='id')

merged_inner.drop('id', axis=1)
# reorder cols to get sampleID at beginning
merged_inner = merged_inner[ ['sampleID']+[col for col in merged_inner.columns if col!='sampleID'] ]

In [30]:
merged_inner.shape
merged_inner.head()
# sample_variant_df.dtypes
# phenotypes.dtypes
# sample_true = []
# for elem in sample_list:
#     sample_true.append(elem in phenotypes['id'])
# print(any(sample_true))

,sampleID,chr10:100048756:A:T,chr10:100048781:C:T,chr10:100048878:T:C,chr10:100054349:G:T,chr10:100057091:G:T,chr10:1000687:C:A,chr10:1000741:C:A,chr10:100152201:G:T,chr10:100152353:C:T,chr10:100154941:T:A,chr10:100164073:G:T,chr10:100164091:C:A,chr10:100167351:A:T,chr10:100167364:T:C,chr10:100167397:C:T,chr10:100167400:G:T,chr10:100174209:T:G,chr10:100174221:C:T,chr10:100178135:G:T,chr10:100178145:C:T,chr10:100178168:C:T,chr10:100183758:G:A,chr10:100183758:G:T,chr10:100183763:G:T,chr10:100183780:G:T,chr10:100183797:G:T,chr10:100185512:A:C,chr10:100185517:T:C,chr10:100185521:A:G,chr10:100193367:G:T,chr10:100194022:T:C,chr10:100194523:T:G,chr10:100204543:G:T,chr10:100204584:T:A,chr10:100204650:G:T,chr10:100205089:G:T,chr10:100207324:A:T,chr10:100209716:G:T,chr10:100218733:G:T,chr10:100219079:G:T,chr10:100220677:C:A,chr10:100235665:A:T,chr10:100235673:A:C,chr10:100235716:C:A,chr10:100235731:C:A,chr10:100238158:G:T,chr10:100238229:G:T,chr10:100250256:T:A,chr10:100256286:A:T,chr10:100260321:T:A,chr10:100280228:T:TATTTCTGGTTTAAGTC,chr10:100286116:G:T,chr10:100288446:C:A,chr10:100290112:T:C,chr10:100294575:T:G,chr10:100294619:G:T,chr10:100294635:A:G,chr10:100295024:T:A,chr10:100295084:G:T,chr10:100295104:A:T,chr10:100295124:C:A,chr10:100296996:T:C,chr10:100297004:G:T,chr10:100297080:A:T,chr10:100298561:C:T,chr10:100299623:G:A,chr10:100299651:ATG:A,chr10:100348333:C:A,chr10:100352414:A:C,chr10:100356765:G:A,chr10:100360740:G:A,chr10:100360745:C:A,chr10:100360754:C:A,chr10:100360757:C:A,chr10:100479052:G:C,chr10:100479054:C:A,chr10:100479085:G:T,chr10:100479956:G:A,chr10:100481078:GATTTTGAT:G,chr10:100482500:T:C,chr10:100489270:T:TGGAAC,chr10:100490203:G:A,chr10:100495509:G:T,chr10:100497275:G:C,chr10:100498070:G:C,chr10:100498133:C:A,chr10:100498774:CAGAG:C,chr10:100499527:CTT:C,chr10:100502317:G:C,chr10:100502363:G:T,chr10:100502486:T:A,chr10:100505396:T:C,chr10:100508001:G:T,chr10:100509089:G:T,chr10:100509441:C:T,chr10:100516216:G:T,chr10:100516872:A:G,chr10:100516951:A:G,chr10:100529386:C:T,...,chrY:21903904:T:A,chrY:2787066:G:T,chrY:2787253:C:A,chrY:2787314:T:A,chrY:2787331:G:T,chrY:2787416:G:T,chrY:2787438:C:T,chrY:2787453:T:A,chrY:2787500:A:T,chrY:2787549:G:T,chrY:2787562:G:T,chrY:2787596:G:T,chrY:2842227:A:ATACACCCGTTAGTTTGT,chrY:2844241:C:A,chrY:2854612:A:T,chrY:2961644:C:A,chrY:2979303:C:A,chrY:2979396:T:A,chrY:2979427:A:T,chrY:2979577:G:T,chrY:2979918:C:A,chrY:5056943:C:A,chrY:5737608:C:A,chrY:6868264:G:A,chrY:6868348:CTGGG:C,chrY:6868354:T:TCCCA,chrY:6868356:T:A,chrY:6868360:G:T,chrY:7070862:G:T,chrY:7080808:C:A,chrY:7085951:C:A,id,Sex_numeric,age,SmokingStatus,ever_smoked,age2,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Coronary_Artery_Disease_SOFT,Prev_Coronary_Artery_Disease_SOFT,Incd_Coronary_Artery_Disease_SOFT,Coronary_Artery_Disease_SOFT_FollowUp,Death,Prev_Death,Incd_Death,Death_FollowUp,Diabetes_All,Prev_Diabetes_All,Incd_Diabetes_All,Diabetes_All_FollowUp,Diabetes_Type_1,Prev_Diabetes_Type_1,Incd_Diabetes_Type_1,Diabetes_Type_1_FollowUp,Diabetes_Type_2,Prev_Diabetes_Type_2,Incd_Diabetes_Type_2,Diabetes_Type_2_FollowUp,Heart_Failure,Prev_Heart_Failure,Incd_Heart_Failure,Heart_Failure_FollowUp,Hypercholesterolemia,Prev_Hypercholesterolemia,Incd_Hypercholesterolemia,Hypercholesterolemia_FollowUp,Hypertension,Prev_Hypertension,Incd_Hypertension,Hypertension_FollowUp,CHIP_Gene,AD,AF,hasCHIP,AML,Prev_AML,Incd_AML,AML_FollowUp,MPN,Prev_MPN,Incd_MPN,MPN_FollowUp,Platelet count,Platelet crit,Mean platelet volume,Platelet distribution width,All_Pneumonia,AgeAt_All_Pneumonia,Incd_All_Pneumonia,All_Pneumonia_FollowUp,Prev_All_Pneumonia
0,1000113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1000113,1,63,0.0,0,3969,-13.4728,5.53740,-2.160480,1.208250,-3.98845,-0.436352,-0.639480,-4.37838,2.542470,2.549570,0,0.0,0.

In [31]:
# compression_opts = dict(method='gzip', archive_name='sample_variant_phenos_leuk_somatic_v2.csv')  
# merged_inner.to_csv('sample_var_phenos_leuk_somatic_v2.csv.gz', 
#                     chunksize=1000,
#                     header=True,
#                     index=False, 
#                     compression=compression_opts)  

In [32]:
merged_inner.to_feather('sample_var_phenos_all_v2.feather')



In [33]:
merged_inner.head()

,sampleID,chr10:100048756:A:T,chr10:100048781:C:T,chr10:100048878:T:C,chr10:100054349:G:T,chr10:100057091:G:T,chr10:1000687:C:A,chr10:1000741:C:A,chr10:100152201:G:T,chr10:100152353:C:T,chr10:100154941:T:A,chr10:100164073:G:T,chr10:100164091:C:A,chr10:100167351:A:T,chr10:100167364:T:C,chr10:100167397:C:T,chr10:100167400:G:T,chr10:100174209:T:G,chr10:100174221:C:T,chr10:100178135:G:T,chr10:100178145:C:T,chr10:100178168:C:T,chr10:100183758:G:A,chr10:100183758:G:T,chr10:100183763:G:T,chr10:100183780:G:T,chr10:100183797:G:T,chr10:100185512:A:C,chr10:100185517:T:C,chr10:100185521:A:G,chr10:100193367:G:T,chr10:100194022:T:C,chr10:100194523:T:G,chr10:100204543:G:T,chr10:100204584:T:A,chr10:100204650:G:T,chr10:100205089:G:T,chr10:100207324:A:T,chr10:100209716:G:T,chr10:100218733:G:T,chr10:100219079:G:T,chr10:100220677:C:A,chr10:100235665:A:T,chr10:100235673:A:C,chr10:100235716:C:A,chr10:100235731:C:A,chr10:100238158:G:T,chr10:100238229:G:T,chr10:100250256:T:A,chr10:100256286:A:T,chr10:100260321:T:A,chr10:100280228:T:TATTTCTGGTTTAAGTC,chr10:100286116:G:T,chr10:100288446:C:A,chr10:100290112:T:C,chr10:100294575:T:G,chr10:100294619:G:T,chr10:100294635:A:G,chr10:100295024:T:A,chr10:100295084:G:T,chr10:100295104:A:T,chr10:100295124:C:A,chr10:100296996:T:C,chr10:100297004:G:T,chr10:100297080:A:T,chr10:100298561:C:T,chr10:100299623:G:A,chr10:100299651:ATG:A,chr10:100348333:C:A,chr10:100352414:A:C,chr10:100356765:G:A,chr10:100360740:G:A,chr10:100360745:C:A,chr10:100360754:C:A,chr10:100360757:C:A,chr10:100479052:G:C,chr10:100479054:C:A,chr10:100479085:G:T,chr10:100479956:G:A,chr10:100481078:GATTTTGAT:G,chr10:100482500:T:C,chr10:100489270:T:TGGAAC,chr10:100490203:G:A,chr10:100495509:G:T,chr10:100497275:G:C,chr10:100498070:G:C,chr10:100498133:C:A,chr10:100498774:CAGAG:C,chr10:100499527:CTT:C,chr10:100502317:G:C,chr10:100502363:G:T,chr10:100502486:T:A,chr10:100505396:T:C,chr10:100508001:G:T,chr10:100509089:G:T,chr10:100509441:C:T,chr10:100516216:G:T,chr10:100516872:A:G,chr10:100516951:A:G,chr10:100529386:C:T,...,chrY:21903904:T:A,chrY:2787066:G:T,chrY:2787253:C:A,chrY:2787314:T:A,chrY:2787331:G:T,chrY:2787416:G:T,chrY:2787438:C:T,chrY:2787453:T:A,chrY:2787500:A:T,chrY:2787549:G:T,chrY:2787562:G:T,chrY:2787596:G:T,chrY:2842227:A:ATACACCCGTTAGTTTGT,chrY:2844241:C:A,chrY:2854612:A:T,chrY:2961644:C:A,chrY:2979303:C:A,chrY:2979396:T:A,chrY:2979427:A:T,chrY:2979577:G:T,chrY:2979918:C:A,chrY:5056943:C:A,chrY:5737608:C:A,chrY:6868264:G:A,chrY:6868348:CTGGG:C,chrY:6868354:T:TCCCA,chrY:6868356:T:A,chrY:6868360:G:T,chrY:7070862:G:T,chrY:7080808:C:A,chrY:7085951:C:A,id,Sex_numeric,age,SmokingStatus,ever_smoked,age2,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Coronary_Artery_Disease_SOFT,Prev_Coronary_Artery_Disease_SOFT,Incd_Coronary_Artery_Disease_SOFT,Coronary_Artery_Disease_SOFT_FollowUp,Death,Prev_Death,Incd_Death,Death_FollowUp,Diabetes_All,Prev_Diabetes_All,Incd_Diabetes_All,Diabetes_All_FollowUp,Diabetes_Type_1,Prev_Diabetes_Type_1,Incd_Diabetes_Type_1,Diabetes_Type_1_FollowUp,Diabetes_Type_2,Prev_Diabetes_Type_2,Incd_Diabetes_Type_2,Diabetes_Type_2_FollowUp,Heart_Failure,Prev_Heart_Failure,Incd_Heart_Failure,Heart_Failure_FollowUp,Hypercholesterolemia,Prev_Hypercholesterolemia,Incd_Hypercholesterolemia,Hypercholesterolemia_FollowUp,Hypertension,Prev_Hypertension,Incd_Hypertension,Hypertension_FollowUp,CHIP_Gene,AD,AF,hasCHIP,AML,Prev_AML,Incd_AML,AML_FollowUp,MPN,Prev_MPN,Incd_MPN,MPN_FollowUp,Platelet count,Platelet crit,Mean platelet volume,Platelet distribution width,All_Pneumonia,AgeAt_All_Pneumonia,Incd_All_Pneumonia,All_Pneumonia_FollowUp,Prev_All_Pneumonia
0,1000113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1000113,1,63,0.0,0,3969,-13.4728,5.53740,-2.160480,1.208250,-3.98845,-0.436352,-0.639480,-4.37838,2.542470,2.549570,0,0.0,0.

In [35]:
test = pd.read_feather('sample_var_phenos_leuk_somatic_topmed_v2.feather')
test.shape

(38070, 27497)